In [2]:
from faker import Faker
import csv
import random
from datetime import datetime, timedelta

fake = Faker()

# Specify latitude and longitude ranges for the campus area
latitude_range = (39.9472, 39.9566)
longitude_range = (-75.1911, -75.2033)

# Generate fake user data
users = []

for user_id in range(1, 41):  # Generate 40 unique user IDs
    interests = random.sample(['Data Science', 'Art', 'Networking', 'Consultancy'], 3)  # Randomly select 3 unique interests for each user
    
    for _ in range(random.randint(5, 6)):  # Generate 5-6 timestamps for each user
        timestamp = fake.date_time_between(start_date='-1d', end_date='now')  # Generate timestamp within the last day
        latitude = round(random.uniform(latitude_range[0], latitude_range[1]), 6)
        longitude = round(random.uniform(longitude_range[0], longitude_range[1]), 6)
        
        user_data = {
            'UserID': user_id,
            'Timestamp': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
            'Latitude': latitude,
            'Longitude': longitude,
            'Interest1': interests[0],
            'Interest2': interests[1],
            'Interest3': interests[2]
        }
        
        users.append(user_data)

# Write fake data to CSV file
with open('fake_users.csv', 'w', newline='') as csvfile:
    fieldnames = ['UserID', 'Timestamp', 'Latitude', 'Longitude', 'Interest1', 'Interest2', 'Interest3']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for user in users:
        writer.writerow(user)


In [5]:
import pandas as pd
from shapely.geometry import Point

# Read the CSV file into a DataFrame
df = pd.read_csv("fake_users.csv")

# Function to create Point geometry in CRS 4326 from latitude and longitude
def create_point_geometry(row):
    lon, lat = row["Longitude"], row["Latitude"]
    return Point(lon, lat)

# Apply the function to create Point geometries
df["geometry"] = df.apply(create_point_geometry, axis=1)

# Optional: Drop the original latitude and longitude columns if needed
#df = df.drop(columns=["latitude_column_name", "longitude_column_name"])

# Save the DataFrame to a new CSV file
df.to_csv("fake_users_geom.csv", index=False)
